### QED-C Application-Oriented Benchmarks - Hamiltonian Simulation with HamLib - Observables (Customizable)

The notebook contains specific examples for the HamLib-based Hamiltonian Simulation benchmark program.
Configure and run the cell below with the desired execution settings.
Then configure and run the remaining cell(s), each one a variation of this benchmark.

Note: This set of benchmarks surfaces the series of second-level functions used to benchmark HamLib observables.
This is a WORK-IN-PROGRESS and is provided to enable experimentation with alternative techniques for computing observables.


In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from math import sin, cos, pi
import time

# Configure module paths
import sys
sys.path.insert(1, "_common")
sys.path.insert(1, "qiskit")

# Import HamLib helper functions (from _common)
import hamlib_utils

# Import Hamlib Simulation kernel (from qiskit)
import hamlib_simulation_kernel

# Import Observable helper functions
import observables

#### for executing circuits to compute observables ...

# Import Qiskit and Qiskit Pauli operator classes
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit import QuantumCircuit

# Initialize simulator backend
from qiskit_aer import Aer
backend = Aer.get_backend('qasm_simulator')


### Configure Options for Execution

In [3]:
# Qubit width of the Hamiltonian
num_qubits = 6

# Parameters of Trotterized simulation
K = 1
t = 0.05

# for debugging Hamlib helper functions
hamlib_simulation_kernel.verbose = False
hamlib_utils.verbose = False

# for debugging observables module
observables.verbose_circuits = False


### Specify a Hamiltonian for Testing
Choose a Hamiltonian from HamLib or create a custom Hamiltonian


In [6]:
# Specify the desired Hamiltonian from HamLib 
# Set the hamiltonian_name: 'TFIM', 'Fermi-Hubbard-1D', 'Bose-Hubbard-1D', 'Heisenberg' or 'Max3Sat'

hamiltonian_name = 'TFIM'

print(f"... Hamiltonian name = {hamiltonian_name}")

########### HamLib Hamiltonians

if hamiltonian_name == 'TFIM':
    hamiltonian_params = { "1D-grid": "pbc", "h": 2 }

if hamiltonian_name == 'Heisenberg':
    hamiltonian_params = { "1D-grid": "pbc", "h": 2 }
    
if hamiltonian_name == 'Fermi-Hubbard-1D':
    hamiltonian_params = { "1D-grid": "pbc", "enc": "bk", "U":12 }

if hamiltonian_name == 'Bose-Hubbard-1D':
    hamiltonian_params = { "1D-grid": "nonpbc", "enc": "gray", "U":10 }

if hamiltonian_name == 'Max3Sat':
    hamiltonian_params = { "ratio": "2", "rinst": "02" }

if hamiltonian_name == 'chemistry/electronic/standard/H2':
    hamiltonian_params = { "ham_BK": '' }

# load the HamLib file for the given hamiltonian name
hamlib_utils.load_hamlib_file(filename=hamiltonian_name)

# return a sparse Pauli list of terms queried from the open HamLib file
sparse_pauli_terms, dataset_name = hamlib_utils.get_hamlib_sparsepaulilist(num_qubits=num_qubits, params=hamiltonian_params)
print(f"... dataset_name = {dataset_name}")
print(f"... sparse_pauli_terms = \n{sparse_pauli_terms}")

print("")


... Hamiltonian name = TFIM
... dataset_name = graph-1D-grid-pbc-qubitnodes_Lx-6_h-2
... sparse_pauli_terms = 
[({0: 'Z', 1: 'Z'}, (1+0j)), ({0: 'Z', 5: 'Z'}, (1+0j)), ({1: 'Z', 2: 'Z'}, (1+0j)), ({5: 'Z', 4: 'Z'}, (1+0j)), ({2: 'Z', 3: 'Z'}, (1+0j)), ({3: 'Z', 4: 'Z'}, (1+0j)), ({0: 'X'}, (2+0j)), ({1: 'X'}, (2+0j)), ({5: 'X'}, (2+0j)), ({2: 'X'}, (2+0j)), ({3: 'X'}, (2+0j)), ({4: 'X'}, (2+0j))]



### Create a Hamiltonian Simulation Circuit
Initialize the circuit to a known initial state, append 'K' Trotter steps for total time 't', followed by basis rotation gates.

In [7]:
# create Trotterized evolution circuit for HamLib Hamiltonians

init_state = "checkerboard"

qc, bitstring = hamlib_simulation_kernel.HamiltonianSimulation(
    num_qubits=num_qubits, 
    ham_op=sparse_pauli_terms,
    K=K, t=t,
    init_state = init_state,
    append_measurements = False,
    method = 1, 
)

print(f"... Trotterized Circuit, K={K}, t={t}")
if num_qubits < 11:
    if hamiltonian_name != '':
        hamlib_simulation_kernel.kernel_draw(qc, method=1)
    else:
        print(qc)
else:
    print("  ... circuit is too large to draw.")

print("")


... Trotterized Circuit, K=1, t=0.05
Sample Circuit:
  H = [({0: 'Z', 1: 'Z'}, (1+0j)), ({0: 'Z', 5: 'Z'}, (1+0j)), ({1: 'Z', 2: 'Z'}, (1+0j)), ({5: 'Z', 4: 'Z'}, (1+0j)), ({2: 'Z', 3: 'Z'}, (1+0j)), ({3: 'Z', 4: 'Z'}, (1+0j)), ({0: 'X'}, (2+0j)), ({1: 'X'}, (2+0j)), ({5: 'X'}, (2+0j)), ({2: 'X'}, (2+0j)), ({3: 'X'}, (2+0j)), ({4: 'X'}, (2+0j))]
     ┌────────┐ ░ ┌───────────────┐ ░ 
q_0: ┤0       ├─░─┤0              ├─░─
     │        │ ░ │               │ ░ 
q_1: ┤1       ├─░─┤1              ├─░─
     │        │ ░ │               │ ░ 
q_2: ┤2       ├─░─┤2              ├─░─
     │  Neele │ ░ │  e^-iHt(0.05) │ ░ 
q_3: ┤3       ├─░─┤3              ├─░─
     │        │ ░ │               │ ░ 
q_4: ┤4       ├─░─┤4              ├─░─
     │        │ ░ │               │ ░ 
q_5: ┤5       ├─░─┤5              ├─░─
     └────────┘ ░ └───────────────┘ ░ 
  Initial State Neele:
     ┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘
q_3: ─────
     ┌───┐
q_4: ┤ X ├
     └───┘
q

### Compute Energy of the Hamiltonian - Unoptimized

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [9]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = False

N = 3
print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
        
    # Estimate expectation for the Ham terms using the circuit qc
    total_energy, term_contributions = observables.estimate_expectation_value(backend, qc, sparse_pauli_terms, 
                                             use_commuting_groups=use_commuting_groups, num_shots=10000)
    print(f"\nTotal Energy: {total_energy}")
    print(f"Term Contributions: {term_contributions}")
    
print("")


************ Compute energy of the Hamiltonian 3 times

Total Energy: (-5.7842+0j)
Term Contributions: {'ZZIIII': -0.9624, 'ZIIIIZ': -0.959, 'IZZIII': -0.9592, 'IIIIZZ': -0.963, 'IIZZII': -0.9586, 'IIIZZI': -0.9564, 'XIIIII': -0.0038, 'IXIIII': -0.0032, 'IIIIIX': -0.006, 'IIXIII': 0.0074, 'IIIXII': -0.0118, 'IIIIXI': 0.0046}

Total Energy: (-5.7998+0j)
Term Contributions: {'ZZIIII': -0.9616, 'ZIIIIZ': -0.9604, 'IZZIII': -0.9626, 'IIIIZZ': -0.9632, 'IIZZII': -0.9612, 'IIIZZI': -0.9584, 'XIIIII': 0.013, 'IXIIII': 0.011, 'IIIIIX': -0.017, 'IIXIII': -0.015, 'IIIXII': -0.0014, 'IIIIXI': -0.0068}

Total Energy: (-5.7108+0j)
Term Contributions: {'ZZIIII': -0.9614, 'ZIIIIZ': -0.958, 'IZZIII': -0.964, 'IIIIZZ': -0.962, 'IIZZII': -0.96, 'IIIZZI': -0.9558, 'XIIIII': 0.0022, 'IXIIII': 0.0056, 'IIIIIX': 0.0088, 'IIXIII': -0.0002, 'IIIXII': 0.0116, 'IIIIXI': -0.0028}



### Compute Energy of the Hamiltonian - Optimized

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [10]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = True

N = 3
print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
        
    # Estimate expectation for the Ham terms using the circuit qc
    total_energy, term_contributions = observables.estimate_expectation_value(backend, qc, sparse_pauli_terms, 
                                             use_commuting_groups=use_commuting_groups, num_shots=10000)
    print(f"\nTotal Energy: {total_energy}")
    print(f"Term Contributions: {term_contributions}")
    
print("")


************ Compute energy of the Hamiltonian 3 times

Total Energy: (-5.7716+0j)
Term Contributions: {'ZZIIII': -0.9592, 'ZIIIIZ': -0.9616, 'IZZIII': -0.956, 'IIIIZZ': -0.9628, 'IIZZII': -0.961, 'IIIZZI': -0.9658, 'XIIIII': 0.0142, 'IXIIII': -0.0046, 'IIIIIX': -0.0196, 'IIXIII': 0.0006, 'IIIXII': -0.0062, 'IIIIXI': 0.013}

Total Energy: (-5.8668+0j)
Term Contributions: {'ZZIIII': -0.9632, 'ZIIIIZ': -0.9622, 'IZZIII': -0.9612, 'IIIIZZ': -0.9626, 'IIZZII': -0.9618, 'IIIZZI': -0.963, 'XIIIII': -0.0128, 'IXIIII': -0.0216, 'IIIIIX': 0.0032, 'IIXIII': -0.0116, 'IIIXII': 0.0018, 'IIIIXI': -0.0054}

Total Energy: (-5.727200000000002+0j)
Term Contributions: {'ZZIIII': -0.96, 'ZIIIIZ': -0.9614, 'IZZIII': -0.9594, 'IIIIZZ': -0.9626, 'IIZZII': -0.956, 'IIIZZI': -0.9594, 'XIIIII': 0.013, 'IXIIII': -0.0052, 'IIIIIX': 0.0094, 'IIXIII': 0.0046, 'IIIXII': -0.0034, 'IIIIXI': -0.0026}



### Compute expectation value of other observables.
Note that this only works for the Ising model.

In [7]:
# Define additional Hamiltonian terms for other Ising observables 
H_terms_spin_correlation = observables.swap_pauli_list([(0.2,'IIIIZZ'), (0.2,'IIIZZI'), (0.2,'IIZZII'), (0.2,'IZZIII'), (0.2,'ZZIIII')])
H_terms_magnetization = observables.swap_pauli_list([(1,'IIIIIZ'), (1,'IIIIZI'), (1,'IIIZII'), (1,'IIZIII'), (1,'IZIIII'), (1, 'ZIIIII')])

spin_correlation = observables.calculate_expectation_from_contributions(term_contributions, H_terms_spin_correlation)
print(spin_correlation)

magnetization = observables.calculate_expectation_from_contributions(term_contributions, H_terms_magnetization)
print(magnetization)


-0.96048
WARN: term not found in term_contributions: IIIIIZ
WARN: term not found in term_contributions: IIIIZI
WARN: term not found in term_contributions: IIIZII
WARN: term not found in term_contributions: IIZIII
WARN: term not found in term_contributions: IZIIII
WARN: term not found in term_contributions: ZIIIII
0
